## 증권리포트 크롤링

- 매일경제증권센터 증권리포트 항목 크롤링
- 검색어, 검색할 maxPage 설정

In [75]:
import requests
from bs4 import BeautifulSoup
from urllib import parse
import re

#증권리포트 제목, 내용, url을 분리, 저장하는 함수
def get_report_inf(url):
    headers = {'Accept-Encoding': 'deflate'}
    res = requests.get(url=url, headers=headers)
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser', from_encoding='euc-kr')#, from_encoding="euc-kr"
    title = soup.find('h1', {'class':'top_title'}).text
    text = soup.find('div', {'class':'art_txt'}).text
    report_data = {}
    report_data['title'] = title.strip()  # 기사 제목
    text = text.replace('googletag','<').replace(');','>').replace('\r','').replace('\n','').replace('\t','')
    text = text.replace('[연합뉴스]Copyrights ⓒ 연합뉴스. 무단 전재 및 재배포금지window.jQuery || document.write','')
    report_data['text'] = re.sub('<.+?>','',text,0).strip()
    #text = text.replace("\r", " ").("\t", " ").strip()
    #report_data['text'] = text.replace("<br>", " ").strip()  # 기사 내용에서 줄바꿈을 지우고, 좌우 공백 제거
    report_data['url'] = url  # 기사 링크
    
    return report_data

In [76]:
#증권리포트 크롤링 data를 반환하는 함수

def get_report_list(page=0  # 뉴스 리스트 페이지
                      ):
    
    
    url='https://www.mk.co.kr/news/business/electonic/?page={page}'  #뉴스>기업>전기,전자,통신
    
    res = requests.get(url.format(page=page))    
    
    # BeautifulSoup을 이용해 웹 html을 파싱
    soup = BeautifulSoup(res.content, 'html.parser')
    #print(soup)
    
    # html 구조에서 해당 기사로 접근할 수 있는 url을 추출    
    div = soup.find('div', {'class':'list_area'})
    alinks = div.findAll('dt', {'class':'tit'})
    #print(alinks)
    report_list = []
    
    for alink in alinks:
        #alink['href'] = alink['href'].replace("./", "http://vip.mk.co.kr/newSt/news/")
        report_url = alink.find('a')['href']
        #print(report_url)
        report_list.append(get_report_inf(report_url))
    
    return report_list

In [77]:
#증권리포트 크롤링 실행
articles = []
for num in range(0, 148):
    articles.extend(get_report_list(page=num))

print("Total articles: {:,}".format(len(articles)))
print("="*20, "Article sample", "="*20)
print(articles[1])

Total articles: 3,700
==================== Article sample ====================
{'title': "LG 코드제로 A9 광고, '셜록' 효과에 조회수 500만 돌파", 'text': 'LG전자 코드제로 \'클린 홈즈\' 광고LG전자는 영국 드라마 \'셜록\'(Sherlock)에서 아이디어를 얻어 제작한 프리미엄 무선청소기 \'코드제로 A9\' 광고 영상이 2주 만에 유튜브 조회 수 500만회를 기록했다고 9일 밝혔다. 이 \'클린 홈즈\' 광고는 청소 전문가로 변신한 셜록 홈즈와 왓슨 박사가 코드제로 A9로 사건 해결 실마리를 얻고, 물걸레 흡입구와 미세먼지 99.999% 차단, 교체형 듀얼 배터리 등 차별점을 소개하는 내용이다.LG전자는 원작에 삽입된 곡 \'The Game in On\'을 광고에 활용하기 위해 제작사인 하츠우드필름과 저작권 사용 계약을 맺을 정도로 공을 들였다.짧은 영화 같은 영상 광고를 본 네티즌들은 호평을 보내고 있다고 LG전자는 전했다.LG전자는 지난해 11월 먼지흡입에 더해 물걸레 청소까지 동시에 할 수 있는 코드제로 A9을 출시했다. ("")', 'url': 'https://www.mk.co.kr/news/business/view/2019/09/714027/'}


## Text preprocessing

- 형태소 분류작업 (정규화, 어근화작업도 수행)

In [83]:
from konlpy.tag import Okt
from collections import Counter
# https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/
tagger = Okt()  # Twitter 형태소 분석기 (Open Korean Text)


#키워드 추출 조건 설정
#input - 키워드로 포함하고자하는 품사 입력, 키워드로 포함하지 않을 금지단어 입력
#output - 기사갯수 100단위로 출력, 시data 첫번째 기사항목 출력
words_list = []
vocab = Counter() # 글자를 숫자로 만들기 위함
tag_set = set(['Noun', 'Alpha'])  # 키워드로 포함하고자 하는 품사 (명사, 알파벳)
stopwords = set(['document', 'jQuery', 'write', 'window', '매일경제', 'Copyrights', 'write', '연합뉴스'])  # 키워드로 포함하지 않을 금지 단어 -> 하면서 포함되도록..
#stopwords.add(searchword)  #검색어 자체가 키워드에 포함되지 않게한다


for i, article in enumerate(articles):
        
    words = tagger.pos(article['text'], norm=True, stem=True) #정규화, 어근화도 진행
    tmp_words = []

    for w, t in words:
        if t in tag_set and len(w) > 1 and w not in stopwords:
            tmp_words.append(w)
    vocab.update(tmp_words)
    words_list.append((tmp_words, article))
    


vocab = sorted([w for w, freq in vocab.most_common(100000)]) # voca size
word2id = {w: i for i, w in enumerate(vocab)}  # {"SK": 0, "로봇": 1, "딥러닝": 2, ...}

new_words_list = []
for words, article in words_list:
    words = [w for w in words if w in word2id]  # vocab에 있는 단어만 사용
    # 기사 내 서로 다른 단어 개수가 5개 이상인 기사만 사용    
    if len(words) > 5:
        new_words_list.append((words, article))

## Build term-document matrix

In [84]:
#input - words_list, vocab
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd


print('기사 갯수 : ' ,len(words_list), ' ', len(new_words_list))
print('vocab 갯수 : ' ,len(vocab))
dtype = [('index', np.float32), ('value', np.float32)]
term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=np.float32) # 메트릭스 만듬
#term_doc_matrix = np.zeros((len(words_list), len(vocab)), dtype=dtype) # 메트릭스 만듬
for i, (words, article) in enumerate(new_words_list):
#for i, (words, article) in enumerate(words_list_filtering):
    for word in words:
        term_doc_matrix[i, word2id[word]] += 1
        
term_doc_matrix = TfidfTransformer().fit_transform(term_doc_matrix) #써이키런 -> 온갖 라이버러리 다 있다
print("Term document matrix shape:", term_doc_matrix.shape)
print(term_doc_matrix)

기사 갯수 :  3700   3606
vocab 갯수 :  21542
Term document matrix shape: (3700, 21542)
  (0, 18664)	0.27457447904196075
  (0, 18077)	0.12103807468996577
  (0, 17927)	0.15584111914672313
  (0, 17673)	0.2888495221871345
  (0, 16036)	0.15237524054329923
  (0, 12747)	0.20537690501526942
  (0, 10640)	0.10151272178310754
  (0, 9071)	0.280370978679273
  (0, 8076)	0.13366929001119895
  (0, 6632)	0.12868347183645212
  (0, 5315)	0.179368473008999
  (0, 5177)	0.265943191322021
  (0, 4595)	0.17540721239338283
  (0, 4555)	0.14110977778333766
  (0, 4332)	0.3558601137336971
  (0, 4074)	0.2311199476936412
  (0, 2496)	0.31464089325390665
  (0, 2478)	0.2861319534242715
  (0, 492)	0.31464089325390665
  (1, 21497)	0.09428129645448952
  (1, 21301)	0.038342472740569074
  (1, 21135)	0.3843700657836088
  (1, 21090)	0.08003999212701703
  (1, 20727)	0.062251482656537675
  (1, 20486)	0.12202830105059896
  :	:
  (3604, 1308)	0.0291217517318658
  (3604, 1126)	0.07803219778716812
  (3604, 877)	0.14337252068349726
  (3604

## Topic modeling

In [85]:
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition
from sklearn.decomposition import NMF
 
K = 8  # topic 개수지정
nmf = NMF(n_components=K, alpha=0.1)  # Non-negative matrix factorization
W = nmf.fit_transform(term_doc_matrix) # 토픽모델링 해준다.
H = nmf.components_

print("W shape:", W.shape)
print("H shape:", H.shape)

W shape: (3700, 8)
H shape: (8, 21542)


In [86]:
top_topic_keywords = []
for k in range(K):
    print("{}th topic".format(k))
    for index in H[k].argsort()[::-1][:15]:
        top_topic_keywords.append((vocab[index], H[k][index]))
        print(vocab[index], end=' ') 
        #print(vocab[index], ' ', H[k][index], end=' ')      
    print("\n", "="*50)
top_topic_keywords = sorted(top_topic_keywords, key=lambda x:x[1], reverse=True)
print(top_topic_keywords)
for x,y in top_topic_keywords:
    print(x)

0th topic
반도체 전자 하이닉스 메모리 생산 시장 업체 삼성 일본 투자 파운드리 낸드 수출 중국 시스템 
1th topic
갤럭시 전자 출시 폴드 노트 삼성 스마트폰 모델 카메라 더블 디스플레이 시리즈 GB 제품 공개 
2th topic
KT 서비스 데이터 기술 유플러스 기업 제공 콘텐츠 고객 AI 클라우드 플랫폼 사업 VR 네트워크 
3th topic
LG 전자 유플러스 고객 듀얼 제품 스크린 체험 제공 스마트폰 가전 김승 배포 무단 금지 
4th topic
이익 영업 매출 실적 대비 감소 증가 김승 동기 분기 기록 뉴스 디지털 사업 기자 
5th topic
SK 텔레콤 하이닉스 ICT 기술 GX 금제 브로드밴드 원금 서비스 고객 미디어 가입자 데이터 제공 
6th topic
계약 공시 금액 디스플레이 공급 매출 체결 규모 장비 제조 연결 해당 Ltd 작년 중국 
7th topic
TV QLED 레드 OLED 전자 시장 디스플레이 인치 삼성 화질 제품 패널 LCD 판매 올해 
[('LG', 2.208389398980074), ('갤럭시', 1.9956800952070173), ('SK', 1.936343881202049), ('TV', 1.8273432843918178), ('반도체', 1.503377458861726), ('계약', 1.3438156521468114), ('KT', 1.2520788190257806), ('이익', 1.2421819047493199), ('영업', 1.205127610050323), ('텔레콤', 1.1951408613086976), ('전자', 0.908824937731961), ('매출', 0.84970884128519), ('서비스', 0.7825220919022633), ('공시', 0.6915545053691361), ('실적', 0.6720819066758752), ('금액', 0.631932855372585), ('전자', 0.6284187890330434), ('QLED', 0.60837723307218

In [ ]:
for k in range(K): # 토픽(15)에 대해 관련 기사를 나열
    print("{}th topic".format(k))
    for index in W[:, k].argsort()[::-1][:5]:
        print(words_list[index][1]['title'])        
    print("\n", "="*80)

## Visualization

In [ ]:
from sklearn.manifold import TSNE

print("W shape:", W.shape)
tsne = TSNE(n_components=2, init='pca', verbose=1) 
W2d = tsne.fit_transform(W)
topic_index = [v.argmax() for v in W]  # 각각의 문서가 어느 topic에 속해 있는지 (hard clustering)


In [ ]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource

output_notebook()

In [ ]:
tools_to_show = "hover,box_zoom,pan,save,reset,wheel_zoom"
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article['title'] for words, article in new_words_list],
    'topic': [str(i) for i in topic_index],
    'color': [Category20[K][i] for i in topic_index]
})



p.circle('x', 'y', source=source, legend='topic', color='color')

p.legend.location = 'top_left'
hover = p.select({'type': HoverTool})
hover.tooltips = [('Topic', '@topic'), ('id', '@id'), ('Article', '@document')]
hover.mode = 'mouse'

show(p)

In [ ]:
#topic과 Keyword 분포, 빈도를 확인할 수 있는 visualization
#jupiter note에서 사용할때 그래프 확인후 interrupt the kernel로 종료 후 다음진행 필요
from sklearn.preprocessing import normalize

def y_to_doc_topic(y):
    n_topics = y.shape[1]
    base = 1 / n_topics
    doc_topic_prob = normalize(y, norm='l1')
    rowsum = doc_topic_prob.sum(axis=1)
    doc_topic_prob[np.where(rowsum == 0)[0]] = base
    return doc_topic_prob


In [ ]:
def components_to_topic_term(components):
    n_terms = components.shape[1]
    base = 1 / n_terms
    topic_term_prob = normalize(components, norm='l1')
    rowsum = topic_term_prob.sum(axis=1)
    topic_term_prob[np.where(rowsum == 0)[0]] = base
    return topic_term_prob

In [ ]:
doc_topic_prob = y_to_doc_topic(W)
topic_term_prob = components_to_topic_term(H)
doc_lengths = np.asarray(term_doc_matrix.sum(axis=1)).reshape(-1)
term_frequency = np.asarray(term_doc_matrix.sum(axis=0)).reshape(-1)

In [ ]:
from pyLDAvis import prepare, show
import pyLDAvis
import pyLDAvis.gensim
import pyLDAvis.sklearn


print(topic_term_prob.shape)
print(len(vocab))
prepared_data = prepare(
    topic_term_prob,
    doc_topic_prob,
    doc_lengths,
    vocab,
    term_frequency,
    R = 20, #num of displayed terms,
    sort_topics = False
)

#display(prepared_data)
show(prepared_data)

## Simple search engine

In [30]:
query = '후성'
query_index = word2id[query]  # 단어의 index

query_vec = term_doc_matrix[:, query_index]  # 각 문서에서 단어에 해당하는 값을 추출
nonzero_doc = query_vec.nonzero()[0]  # 0이 아닌 문서들만 사용
#print('논제로_독', nonzero_doc);
ranks = query_vec.toarray().flatten().argsort()[::-1]
#print('랭크', ranks);



for r in ranks[:10]:
    if r in nonzero_doc:
        print(new_words_list[r][1]['title'], new_words_list[r][1]['url'])


[반도체] 소재 업종, 이유 있는 주가 강세 http://vip.mk.co.kr/newSt/news/news_view2.php?p_page=1&sCode=110&t_uid=6&c_uid=43989&search=[반도체]
[반도체] 고순도 불화수소 이슈 정리 http://vip.mk.co.kr/newSt/news/news_view2.php?p_page=1&sCode=110&t_uid=6&c_uid=44040&search=[반도체]
[반도체] 미중패권전쟁과 한일무역전쟁, 일본의 거센 공격이 가져올 구조적 변화 http://vip.mk.co.kr/newSt/news/news_view2.php?p_page=1&sCode=110&t_uid=6&c_uid=44170&search=[반도체]
[반도체] 반도체 소재 국산화 비중 증가 http://vip.mk.co.kr/newSt/news/news_view2.php?p_page=1&sCode=110&t_uid=6&c_uid=43988&search=[반도체]
